# Version 2 EDA

## TODO - Questions

1. 

## Imports and Global Settings

In [2]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from sqlalchemy import create_engine

sys.path.append('../')
from passkeys import RDS_ENDPOINT, RDS_PASSWORD

# Pandas Settings
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows',1000)
pd.options.display.max_info_columns = 1000
pd.options.display.precision = 5

# Graphing Settings
sns.set_theme()

## Data Inbound

### Database Connection

In [3]:
username = 'postgres'
password = RDS_PASSWORD
endpoint = RDS_ENDPOINT
database = 'nba_betting'
port = '5432'

# Create the connection string
connection_string = f'postgresql+psycopg2://{username}:{password}@{endpoint}:{port}/{database}'

### Database Queries

In [9]:
start_date = "2010-09-01" # September 1st, 2010 

# Game Results
nbastats_game_results_query = f"SELECT * FROM ibd_nba_stats_game_results WHERE game_date >= '{start_date}';"

# Betting Data
covers_query = f"SELECT * FROM covers WHERE date >= '{start_date}';"

# NBA Stats Player Boxscores
nbastats_player_boxscores_traditional_query = f"SELECT * FROM ibd_nba_stats_boxscores_traditional WHERE game_date >= '{start_date}';"
nbastats_player_boxscores_adv_traditional_query = f"SELECT * FROM ibd_nba_stats_boxscores_adv_traditional WHERE game_date >= '{start_date}';"
nbastats_player_boxscores_adv_advanced_query = f"SELECT * FROM ibd_nba_stats_boxscores_adv_advanced WHERE game_date >= '{start_date}';"

# NBA Stats Player Stats
nbastats_player_general_traditional_query = f"SELECT * FROM ibd_nba_stats_player_general_traditional WHERE to_date >= '{start_date}';"
nbastats_player_general_advanced_query = f"SELECT * FROM ibd_nba_stats_player_general_advacned WHERE to_date >= '{start_date}';"

# FiveThirtyEight Player Data
fivethirtyeight_player_query = f"SELECT * FROM covers WHERE to_date >= '{start_date}';"

### Base Tables

In [ ]:
with create_engine(connection_string).connect() as connection:
    # Game Results
    nbastats_game_results = pd.read_sql_query(nbastats_game_results_query, connection, parse_dates=['game_date'])

    # Betting Data
    covers_betting_data = pd.read_sql_query(covers_query, connection, parse_dates=['date'])

    # NBA Stats Player Boxscores
    nbastats_player_boxscores_traditional = pd.read_sql_query(nbastats_player_boxscores_traditional_query,
                                                           connection, parse_dates=['game_date'])
    nbastats_player_boxscores_adv_traditional = pd.read_sql_query(nbastats_player_boxscores_adv_traditional_query,
                                                               connection, parse_dates=['game_date'])
    nbastats_player_boxscores_adv_advanced = pd.read_sql_query(nbastats_player_boxscores_adv_advanced_query,
                                                            connection, parse_dates=['game_date'])

    # NBA Stats Player Stats
    nbastats_player_general_traditional = pd.read_sql_query(nbastats_player_general_traditional_query,
                                                            connection, parse_dates=['to_date'])
    nbastats_player_general_advanced = pd.read_sql_query(nbastats_player_general_advanced_query,
                                                            connection, parse_dates=['to_date'])

    # FiveThirtyEight Player Data
    fivethirtyeight_player = pd.read_sql_query(fivethirtyeight_player_query,
                                            connection, parse_dates=['to_date'])

### Working Tables - **Restart From Here**

In [ ]:
# Game Results
games_df = nbastats_game_results.copy()

# Betting Data
betting_df = covers_betting_data.copy()

# NBA Stats Player Boxscores
trad_box_df = nbastats_player_boxscores_traditional.copy()
tradadv_box_df = nbastats_player_boxscores_adv_traditional.copy()
adv_box_df = nbastats_player_boxscores_adv_advanced.copy()

# NBA Stats Player Stats
stats_trad_df = nbastats_player_general_traditional.copy()
stats_adv_df = nbastats_player_general_advanced.copy()

# FiveThirtyEight Player Data
five38_df = fivethirtyeight_player.copy()

In [ ]:
def print_table_info(df):
    # Print the head(10)
    print("Head (first 10 rows):")
    print(df.head(10))

    # Print the info()
    print("\nInfo:")
    print(df.info(verbose=True, show_counts=True))